In [1]:
# Import dependencies for ml data processing
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

plt.rcParams["font.size"] = 16

In [2]:
df = pd.read_csv('github_user_segmentation/data/raw/raw_df.csv', index_col=0)
df = df.drop(columns='repo_id')
df['event_count'] = df.groupby('actor_id')['id'].transform('count')
df['type'] = df['type'].str.replace('Event', '')
df['type'] = df['type'].str.replace('PullRequest', 'Pull Request')
df['type'] = df['type'].str.replace('RequestReview', 'Request Review')
df['type'] = df['type'].str.replace('ReviewComment', 'Review Comment')
df['type'] = df['type'].str.replace('CommitComment', 'Commit Comment')
df['event_count'] = df.groupby('actor_id')['id'].transform('count')
filtered_df = df[df['event_count'] > 5]

In [3]:
date_string = filtered_df.copy()
date_string[['created_at', 'last_name']] = filtered_df['created_at'].str.split('+', n=1, expand=True)
date_string = date_string.drop(columns=['event_count', 'last_name'])
date_string['created_at'] = pd.to_datetime(date_string['created_at'])

In [6]:
starting_step = 'Start'
start_events = pd.DataFrame({
    'created_at': [pd.Timestamp('2023-03-17 00:00:00')] * date_string['actor_id'].nunique(),
    'actor_id': date_string['actor_id'].unique(),
    'type': [starting_step] * date_string['actor_id'].nunique(),
    'id': date_string['actor_id'].unique() + 100000000000
})

combined_df = pd.concat([start_events, date_string], ignore_index=True)
combined_df = combined_df.sort_values(['actor_id', 'created_at'])
combined_df.to_csv('github_user_segmentation/data/processed/combined_sankey.csv')
output_file = 'github_user_segmentation/data/processed/combined_sankey.h5'
combined_df.to_hdf(output_file, key='data', mode='w')